In [1]:
%%capture
!pip install folium matplotlib mapclassify

In [2]:
import pandas as pd
import geopandas as gpd
import os
import zipfile

FOLDER_PATH = '/content/drive/MyDrive/DSL SMART CITIES'

from google.colab import drive
drive.mount('/content/drive')

COLS_TO_KEEP = ['fclass','geometry']

Mounted at /content/drive


# Import NILs

In [3]:
nil = gpd.read_file(os.path.join(FOLDER_PATH, 'Processed','nil.geojson'))
nil = nil.to_crs(epsg=4326)
nil

,ID_NIL,NIL,area_sqkm,Residenti,geometry
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ..."
1,2,brera,1.637333,18294,"POLYGON ((9.19165 45.46906, 9.19204 45.46937, ..."
2,3,giardini p.ta venezia,0.249637,43,"POLYGON ((9.20090 45.47654, 9.20000 45.47696, ..."
3,4,guastalla,1.547962,15528,"POLYGON ((9.20700 45.46787, 9.20693 45.46853, ..."
4,5,porta vigentina - porta lodovica,1.135196,13548,"POLYGON ((9.20186 45.45238, 9.20078 45.45301, ..."
...,...,...,...,...,...
83,84,parco nord,1.532331,98,"POLYGON ((9.20040 45.52848, 9.20028 45.52846, ..."
84,85,parco delle abbazie,13.733841,364,"POLYGON ((9.21711 45.43187, 9.21531 45.43232, ..."
85,86,parco dei navigli,3.623149,354,"POLYGON ((9.13886 45.42855, 9.13769 45.42947, ..."
86,87,assiano,5.840942,211,"POLYGON ((9.04687 45.46276, 9.04685 45.46278, ..."


# Import Public Transports stop data

In [27]:
underground_stops = gpd.read_file(os.path.join(FOLDER_PATH,'ATM_FERMATE_METRO_MILANO.txt'))
surf_stops = gpd.read_file(os.path.join(FOLDER_PATH,'ATM_FERMATE_SU_MILANO.txt'))

In [28]:
underground_stops['type'] = 'underground'
underground_stops.head()

,id_amat,nome,linee,geometry,type
0,882,TRICOLORE,4,POINT (9.20777 45.46790),underground
1,883,DATEO,4,POINT (9.21845 45.46799),underground
2,884,SUSA,4,POINT (9.22488 45.46807),underground
3,885,ARGONNE,4,POINT (9.23196 45.46813),underground
4,886,STAZIONE FORLANINI,4,POINT (9.23635 45.46431),underground


In [29]:
surf_stops['type'] = 'surface'
surf_stops = surf_stops.rename(columns={"ubicazione": "nome"})
surf_stops.head()

,id_amat,nome,linee,geometry,type
0,10003,"Via Bovisasca, 18 dopo Via C.Battisti",89,POINT (9.14826 45.52691),surface
1,10008,"Via Bovisasca, 15/A prima di Via Baracca",89,POINT (9.14845 45.52675),surface
2,10018,"C.so Sempione, 83 prima di Via E. Filiberto",48,POINT (9.15805 45.48628),surface
3,10021,Viale Alfa Romeo fronte civico 17,561,POINT (9.05271 45.55841),surface
4,10022,Via Mosca prima V.Val Cavallina,63,POINT (9.07122 45.45214),surface


In [30]:
public_transport_stop = pd.concat([underground_stops, surf_stops])
public_transport_stop['type_unique']

,id_amat,nome,linee,geometry,type
0,882,TRICOLORE,4,POINT (9.20777 45.46790),underground
1,883,DATEO,4,POINT (9.21845 45.46799),underground
2,884,SUSA,4,POINT (9.22488 45.46807),underground
3,885,ARGONNE,4,POINT (9.23196 45.46813),underground
4,886,STAZIONE FORLANINI,4,POINT (9.23635 45.46431),underground


# Sjoin

In [31]:
transports_nil = nil.sjoin(public_transport_stop, how='left')
transports_nil

,ID_NIL,NIL,area_sqkm,Residenti,geometry,index_right,id_amat,nome,linee,type
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",1179,11467,"Via S.Sofia, 32 prima di C.so Italia",94,surface
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",1177,11465,"C.so Italia, 34 dopo Via Crocefisso","15,115",surface
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",1185,11473,"Via S.Sofia, 18 prima di Via S.Senatore",94,surface
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",1217,11520,C.so di P.ta Romana fronte 46 altezza P.za S.N...,"16,24,124,153",surface
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",1202,11498,"C.so Italia, 8 prima di Via Barellai","15,115",surface
...,...,...,...,...,...,...,...,...,...,...
87,88,parco bosco in citta',7.834006,690,"POLYGON ((9.07309 45.50604, 9.07304 45.50606, ...",60,10084,Via Novara fronte 201 prima di Via C.na Bellaria,"64,80,423",surface
87,88,parco bosco in citta',7.834006,690,"POLYGON ((9.07309 45.50604, 9.07304 45.50606, ...",55,10079,"Via Cascina Bellaria, 90",64,surface
87,88,parco bosco in citta',7.834006,690,"POLYGON ((9.07309 45.50604, 9.07304 45.50606, ...",51,10075,Via Cascina Bellaria fronte 90,64,surface
87,88,parco bosco in citta',7.834006,690,"POLYGON ((9.07309 45.50604, 9.07304 45.50606, ...",3234,15157,"Via Novara, 451",80,surface


# Groupby

In [53]:
stops_grouped = transports_nil.groupby(['ID_NIL','NIL','area_sqkm','Residenti','type']).size().reset_index(name='count')
stops_grouped

,ID_NIL,NIL,area_sqkm,Residenti,type,count
0,1,duomo,2.341616,16608,surface,74
1,1,duomo,2.341616,16608,underground,6
2,10,stazione centrale - ponte seveso,1.556019,18410,surface,56
3,10,stazione centrale - ponte seveso,1.556019,18410,underground,3
4,11,isola,1.322887,22641,surface,41
...,...,...,...,...,...,...
124,86,parco dei navigli,3.623149,354,surface,2
125,87,assiano,5.840942,211,surface,1
126,88,parco bosco in citta',7.834006,690,surface,10
127,9,porta garibaldi - porta nuova,0.785638,5921,surface,23


In [54]:
# Step 1: Filter and group by type
underground_stops = stops_grouped[stops_grouped['type'] == 'underground'].groupby(['ID_NIL','NIL','area_sqkm','Residenti'])['count'].sum().reset_index()
surface_stops = stops_grouped[stops_grouped['type'] == 'surface'].groupby(['ID_NIL','NIL','area_sqkm','Residenti'])['count'].sum().reset_index()

# Step 2: Merge with the original DataFrame to retain residents information
stops_merged = stops_grouped[['ID_NIL','NIL','area_sqkm','Residenti']].drop_duplicates().merge(underground_stops, on=['ID_NIL','NIL','area_sqkm','Residenti'], how='left').merge(surface_stops, on=['ID_NIL','NIL','area_sqkm','Residenti'], how='left', suffixes=('_underground', '_surface'))

# Step 3: Calculate the number of stops per resident
stops_merged['underground_stops_per_resident'] = stops_merged['count_underground'] / stops_merged['Residenti']
stops_merged['surface_stops_per_resident'] = stops_merged['count_surface'] / stops_merged['Residenti']
stops_merged['underground_stops_per_sqkm'] = stops_merged['count_underground'] / stops_merged['area_sqkm']
stops_merged['surface_stops_per_sqkm'] = stops_merged['count_surface'] / stops_merged['area_sqkm']

# Step 4: Handle any NaN values (if there are NILs without a certain type of stop)
stops_merged.fillna(0, inplace=True)

stops_merged

/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,ID_NIL,NIL,area_sqkm,Residenti,count_underground,count_surface,underground_stops_per_resident,surface_stops_per_resident,underground_stops_per_sqkm,surface_stops_per_sqkm
0,1,duomo,2.341616,16608,6.0,74,0.000361,0.004456,2.562333,31.602113
1,10,stazione centrale - ponte seveso,1.556019,18410,3.0,56,0.000163,0.003042,1.927997,35.989284
2,11,isola,1.322887,22641,2.0,41,0.000088,0.001811,1.511845,30.992818
3,12,maciachini - maggiolina,1.674918,26097,3.0,37,0.000115,0.001418,1.791133,22.090639
4,13,greco - segnano,1.768603,16137,0.0,34,0.000000,0.002107,0.000000,19.224217
...,...,...,...,...,...,...,...,...,...,...
83,85,parco delle abbazie,13.733841,364,0.0,19,0.000000,0.052198,0.000000,1.383444
84,86,parco dei navigli,3.623149,354,0.0,2,0.000000,0.005650,0.000000,0.552006
85,87,assiano,5.840942,211,0.0,1,0.000000,0.004739,0.000000,0.171205
86,88,parco bosco in citta',7.834006,690,0.0,10,0.000000,0.014493,0.000000,1.276486


In [55]:
#stops_merged.to_csv(os.path.join(FOLDER_PATH,'Processed','stops_grouped.csv'))
